# Import

In [13]:
import pandas as pd
import numpy as np

from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

import torch
import tensorflow as tf

import warnings
warnings.filterwarnings(action='ignore')

Init Plugin
Init Graph Optimizer
Init Kernel


# Data Load

In [2]:
train_df = pd.read_csv('/Users/hanhyeongu/Desktop/HG/Dacon/Credit_fraud_detection/data/train.csv').drop('ID',axis=1)
valid_df = pd.read_csv('/Users/hanhyeongu/Desktop/HG/Dacon/Credit_fraud_detection/data/val.csv').drop('ID', axis=1)

# Validation Dataset fraud ratio

In [3]:
val_normal, val_fraud = valid_df['Class'].value_counts()
val_contamination = val_fraud / val_normal
print(f'Validation contamination : [{val_contamination}]')

Validation contamination : [0.0010551491277433877]


# Model Define & Fit

In [4]:
X_train = train_df

In [5]:
# 가설검정: train dataset도 validation dataset과 동일한 비율로 사기거래가 발생했을 것이다. 
model = EllipticEnvelope(
    contamination=val_contamination,
    random_state=42
)

model.fit(X_train)

EllipticEnvelope(contamination=0.0010551491277433877, random_state=42)

# Evaluation

In [6]:
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [7]:
X_val = valid_df.drop(columns=['Class']) # Input Data
y_val = valid_df['Class'] # Label

val_pred = model.predict(X_val) # model prediction
val_pred = get_pred_label(val_pred)
val_score = f1_score(y_val, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(y_val, val_pred))

Validation F1 Score : [0.7980433877878794]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.63      0.57      0.60        30

    accuracy                           1.00     28462
   macro avg       0.81      0.78      0.80     28462
weighted avg       1.00      1.00      1.00     28462

